# Your turn at home!

![](images/dev.gif)


## REST APIs

  * Read an introduction to the Representational State Transfer architectural style in chapter 14 of _"Network Programming with Go"_, see Network_Programming_with_Go_Chap14.pdf.
  * Find a more brief introduction to REST in chapter 7.3 of _Go Web Programming_, see Go_Web_Programming_Chap7_part2.pdf
  * Read the original resource on the Representational State Transfer architectural style, i.e., chapter five of Roy Fieldings dissertation https://www.ics.uci.edu/~fielding/pubs/dissertation/rest_arch_style.htm.
  * Read on two different strategies for versioning REST APIs, in chapter 8.4 of _Go in Practice_, see `Go_in_Practice_chap8_4.pdf`.
    
Get the documents here: https://github.com/datsoftlyngby/soft2018fall-si-teaching-material/blob/master/Resources/session7.zip 


  * [`Network_Programming_with_Go_Chap14.pdf`](../Resources/Network_Programming_with_Go_Chap14.pdf)    
  * [`Go_Web_Programming_Chap7_part2.pdf`](../Resources/Go_Web_Programming_Chap7_part2.pdf)
  * [`Go_in_Practice_chap8_4`](../Resources/Go_in_Practice_chap8_4.pdf)
  * [`Go_Web_Programming_Chap7_part2.pdf`](../Resources/Go_Web_Programming_Chap7_part2.pdf)

# Demo REST API?

Does the following server expose a REST API? If not, which of the REST architectural style properties is violated? Furthermore, if it is not RESTful what would it take to make the computation service RESTful?


```go
package main

import (
	"fmt"
	"io"
	"io/ioutil"
	"log"
	"net/http"
	"strconv"
)

type Stack struct {
	top  *Node
	size int
}

type Node struct {
	value int
	next  *Node
}

func (s *Stack) Peek() int {
	return s.top.value
}

func (s *Stack) Push(val int) {
	s.top = &Node{val, s.top}
	s.size++
}

func (s *Stack) Pop() (val int) {
	if s.size > 0 {
		val, s.top = s.top.value, s.top.next
		s.size--
		return val
	}
	return 0
}

func getNumberFrom(requestBody io.ReadCloser) int {
	body, err := ioutil.ReadAll(requestBody)
	if err != nil {
		log.Println(err.Error())
	}
	number, err := strconv.Atoi(string(body))
	if err != nil {
		log.Println("Ups... cannot get the number")
	}
	return number
}

func pushNumber(res http.ResponseWriter, req *http.Request) {
	if req.Method == "POST" {
		// Test for example with:
		// curl -X POST http://192.168.20.2:8080/push -d '5'
		number := getNumberFrom(req.Body)
		stack.Push(number)
		fmt.Fprint(res, stack.Peek())
	}
}

func addNumber(res http.ResponseWriter, req *http.Request) {
	number := getNumberFrom(req.Body)
	stack.Push(number)
	result := stack.Pop() + stack.Pop()
	stack.Push(result)
	fmt.Fprint(res, result)
}

func subNumber(res http.ResponseWriter, req *http.Request) {
	number := getNumberFrom(req.Body)
	stack.Push(number)
	result := stack.Pop() - stack.Pop()
	stack.Push(result)
	fmt.Fprint(res, result)
}

var stack *Stack

func main() {
	stack = new(Stack)
	stack.Push(0)

	http.HandleFunc("/push", pushNumber)
	http.HandleFunc("/add", addNumber)
	http.HandleFunc("/sub", subNumber)

	http.ListenAndServe(":8080", nil)
}
```

# Demo of a Small Peer-to-Peer (P2P) Network

The code for a single peer-to-peer node, which sends random numbers to random other nodes. Additionally, it sums up all numbers that it receives from other nodes.

```go
package main

import (
	"bytes"
	"encoding/json"
	"errors"
	"fmt"
	"io/ioutil"
	"log"
	"math/rand"
	"net/http"
	"os"
	"strconv"
	"time"
)

type configuration struct {
	Peers []string `json:"peers"`
}

func readConfigFile(path string) configuration {
	file, err := os.Open(path)
	if err != nil {
		log.Fatalf("Cannot find configuration file %s", path)
	}
	defer file.Close()

	conf := new(configuration)
	decoder := json.NewDecoder(file)
	err = decoder.Decode(&conf)
	if err != nil {
		log.Fatalf("Cannot read configuration file %s", path)
	}

	return *conf
}

func ping(res http.ResponseWriter, req *http.Request) {
	// time.Sleep(1 * time.Second)
	fmt.Fprint(res, "Active")
}

func sendRequest(peerURL string) (string, error) {
	client := &http.Client{
		Timeout: time.Duration(2 * time.Second),
	}
	request, _ := http.NewRequest("GET", peerURL, nil)
	request.Header.Set("Accept", "text/plain; charset=utf-8")
	response, err := client.Do(request)

	if err != nil {
		return "", err
	} else {
		body, err := ioutil.ReadAll(response.Body)
		if err != nil {
			log.Println(err.Error())
		}
		return string(body), nil
	}
	return "", errors.New("Something went terribly wrong?")
}

func pingAll(res http.ResponseWriter, req *http.Request) {
	var peersToRemove []string
	for _, peer := range config.Peers {
		peerURL := fmt.Sprintf("%s/state", peer)
		fmt.Println(peerURL)
		msg, err := sendRequest(peerURL)
		if err != nil {
			log.Println(err)
			peersToRemove = append(peersToRemove, peer)
		} else {
			log.Println(msg)
		}
	}

	conf := configuration{
		Peers: peersToRemove,
	}
	confAsBytes, _ := json.Marshal(conf)
	fmt.Fprintln(res, string(confAsBytes))
}

func getNumber(res http.ResponseWriter, req *http.Request) {
	if req.Method == "POST" {
		// Test for example with:
		// curl -X POST http://192.168.20.2:8080/msg -d '5'

		body, err := ioutil.ReadAll(req.Body)
		if err != nil {
			log.Println(err.Error())
		}
		number, err := strconv.Atoi(string(body))
		if err != nil {
			log.Println("Ups... cannot get the number")
		}
		internalSum += number
	} else if req.Method == "GET" {
		fmt.Fprint(res, internalSum)
	}
}

func keepBusy() {
	randomPeerIdx := rand.Intn(len(config.Peers))
	randomNumber := strconv.Itoa(rand.Intn(100))
	fmt.Println("Sending", randomNumber, "to", config.Peers[randomPeerIdx])
	_, err := http.Post(config.Peers[randomPeerIdx]+"/msg", "text/plain", bytes.NewBuffer([]byte(randomNumber)))
	if err != nil {
		log.Println(err)
	}
}

func addPeers(res http.ResponseWriter, req *http.Request) {
	// TODO: One of the exercises
}

func removePeers(res http.ResponseWriter, req *http.Request) {
	// TODO: One of the exercises
}

var pathToConfig = "peers.json"
var config = readConfigFile(pathToConfig)
var internalSum = 0

func main() {
	rand.Seed(time.Now().UnixNano())

	// equivalent to Python's if not os.path.exists(filename)
	if _, err := os.Stat(pathToConfig); os.IsNotExist(err) {
		log.Fatalf("Config file %s does not exist", pathToConfig)
	}
	http.HandleFunc("/state", ping)
	http.HandleFunc("/excludes", pingAll)
	http.HandleFunc("/msg", getNumber)
	http.HandleFunc("/newPeers", addPeers)
	http.HandleFunc("/garbagePeers", removePeers)

	// The following calls the keepBusy func every so and so many seconds
	// see: https://stackoverflow.com/questions/43002163/run-function-only-once-for-specific-time-golang
	ticker := time.NewTicker(time.Duration(rand.Intn(10)) * time.Second)
	go func(ticker *time.Ticker) {
		for {
			select {
			case <-ticker.C:
				keepBusy()
			}
		}
	}(ticker)

	http.ListenAndServe(":8080", nil)
}
```

In the following is a JSON file, which keeps a configuration for a small p2p-network consisting of three nodes.

```json
{
    "peers": ["http://192.168.20.2:8080", 
              "http://192.168.20.3:8080", 
              "http://192.168.20.4:8080"]
}
```


## A Small Peer-to-Peer (P2P) Network with Graceful Shutdown


```go
package main

import (
	"bytes"
	"context"
	"encoding/json"
	"errors"
	"fmt"
	"io/ioutil"
	"log"
	"math/rand"
	"net/http"
	"os"
	"os/signal"
	"strconv"
	"sync"
	"time"
)

type configuration struct {
	Peers []string `json:"peers"`
}

func readConfigFile(path string) configuration {
	file, err := os.Open(path)
	if err != nil {
		log.Fatalf("Cannot find configuration file %s", path)
	}
	defer file.Close()

	conf := new(configuration)
	decoder := json.NewDecoder(file)
	err = decoder.Decode(&conf)
	if err != nil {
		log.Fatalf("Cannot read configuration file %s", path)
	}

	return *conf
}

func ping(res http.ResponseWriter, req *http.Request) {
	fmt.Fprint(res, "Active")
}

func sendRequest(peerURL string) (string, error) {
	client := &http.Client{
		Timeout: time.Duration(2 * time.Second),
	}
	request, _ := http.NewRequest("GET", peerURL, nil)
	request.Header.Set("Accept", "text/plain; charset=utf-8")
	response, err := client.Do(request)

	if err != nil {
		return "", err
	} else {
		body, err := ioutil.ReadAll(response.Body)
		if err != nil {
			log.Println(err.Error())
		}
		return string(body), nil
	}
	return "", errors.New("Something went terribly wrong?")
}

func pingAll(res http.ResponseWriter, req *http.Request) {
	var peersToRemove []string
	for _, peer := range config.Peers {
		peerURL := fmt.Sprintf("%s/state", peer)
		fmt.Println(peerURL)
		msg, err := sendRequest(peerURL)
		if err != nil {
			log.Println(err)
			peersToRemove = append(peersToRemove, peer)
		} else {
			log.Println(msg)
		}
	}

	conf := configuration{
		Peers: peersToRemove,
	}
	confAsBytes, _ := json.Marshal(conf)
	fmt.Fprintln(res, string(confAsBytes))
}

func getNumber(res http.ResponseWriter, req *http.Request) {
	if req.Method == "POST" {
		// Test for example with:
		// curl -X POST http://192.168.20.2:8080/msg -d '5'

		body, err := ioutil.ReadAll(req.Body)
		if err != nil {
			log.Println(err.Error())
		}
		number, err := strconv.Atoi(string(body))
		if err != nil {
			log.Println("Ups... cannot get the number")
		}
		internalSum += number
	} else if req.Method == "GET" {
		fmt.Fprint(res, internalSum)
	}
}

func keepBusy() {
	randomPeerIdx := rand.Intn(len(config.Peers))
	randomNumber := strconv.Itoa(rand.Intn(100))
	fmt.Println("Sending", randomNumber, "to", config.Peers[randomPeerIdx])
	_, err := http.Post(config.Peers[randomPeerIdx]+"/msg", "text/plain", bytes.NewBuffer([]byte(randomNumber)))
	if err != nil {
		log.Println(err)
	}
}

func addPeers(res http.ResponseWriter, req *http.Request) {
	// TODO: One of the exercises
}

func removePeers(res http.ResponseWriter, req *http.Request) {
	// TODO: One of the exercises
}

var pathToConfig = "peers.json"
var config = readConfigFile(pathToConfig)
var internalSum = 0

func main() {
	rand.Seed(time.Now().UnixNano())

	// equivalent to Python's if not os.path.exists(filename)
	if _, err := os.Stat(pathToConfig); os.IsNotExist(err) {
		log.Fatalf("Config file %s does not exist", pathToConfig)
	}
	http.HandleFunc("/state", ping)
	http.HandleFunc("/excludes", pingAll)
	http.HandleFunc("/msg", getNumber)
	http.HandleFunc("/newPeers", addPeers)
	http.HandleFunc("/garbagePeers", removePeers)

	// The following calls the keepBusy func every so and so many seconds
	// see: https://stackoverflow.com/questions/43002163/run-function-only-once-for-specific-time-golang
	ticker := time.NewTicker(time.Duration(rand.Intn(10)) * time.Second)
	go func(ticker *time.Ticker) {
		for {
			select {
			case <-ticker.C:
				keepBusy()
			}
		}
	}(ticker)

	// Create a new server and set timeout values.
	server := http.Server{
		Addr: ":8080",
	}

	// We want to report the listener is closed.
	var wg sync.WaitGroup
	wg.Add(1)

	// Start the listener.
	go func() {
		log.Println("listener : Listening on localhost:3000")
		log.Println("listener :", server.ListenAndServe())
		wg.Done()
	}()

	// Listen for an interrupt signal from the OS. Use a buffered
	// channel because of how the signal package is implemented.
	osSignals := make(chan os.Signal, 1)
	signal.Notify(osSignals, os.Interrupt)

	// Wait for a signal to shutdown.
	<-osSignals

	// Create a context to attempt a graceful 5 second shutdown.
	const timeout = 5 * time.Second
	ctx, cancel := context.WithTimeout(context.Background(), timeout)
	defer cancel()

	// Attempt the graceful shutdown by closing the listener and
	// completing all inflight requests.
	if err := server.Shutdown(ctx); err != nil {
		log.Printf("shutdown : Graceful shutdown did not complete in %v : %v", timeout, err)

		// Looks like we timedout on the graceful shutdown. Kill it hard.
		if err := server.Close(); err != nil {
			log.Printf("shutdown : Error killing server : %v", err)
		}
	}

	// Wait for the listener to report it is closed.
	wg.Wait()
	log.Println("main : Completed")
}
```



# Your turn!

![](images/your_turn.gif)

# First Hour

  * Modify the HTTP server from the first code example so that it's API is RESTful.
  * Write a small web service, which via a RESTful API, provides weather and sunrise information information for the location of a given Github user.
    - For example: running `curl -X POST http://<your_host>/v1/weatherfor -d {"username":"HelgeCPH"}`
    shall return 
    ```json
    {"coord": {"lon":12.57,"lat":55.69},
     "weather":[{"main":"Mist"}]
     "temp":12.21,
     "pressure":1021,
     "humidity":93,
     "temp_min":10,
     "temp_max":14,
     "sunrise":"Tue Oct  9 07:29:42 CEST 2018",
     "sunset":"Tue Oct  9 18:22:49 CEST 2018"
    }
    ```

    That is, in general, the call is as in the following: `curl http://<your_host>/v1/describe -d {"username": <username>}`
  
    - Make use of the Github API, see https://developer.github.com/v3/, in particular of https://developer.github.com/v3/users/#get-a-single-user
    - and the Open Weather Map API, see https://openweathermap.org/current and https://openweathermap.org/appid#get
      - **OBS** you need to register at their service and you need to create an API token to use this service. Then you can export this key into an environment variable (`export OWM_API_KEY="<your_api_key>"`), so that you can run for example: 
        ~~~bash
$ curl "http://api.openweathermap.org/data/2.5/weather?q=Copenhagen,dk&units=metric&appid=${OWM_API_KEY}"
~~~
  * Now, migrate your API to a new version in which you can perform the same query. But instead of sending the user name and the repository name in the payload, allow queries of the form of: `http://<your_host>/v2/<username>`
  
  
  
# Second Hour
  
  * Build a P2P application with the help of (REST) API calls.
    - Create an endpoint to register a single new node or many nodes to your network. The mode of operation is, that when a new node gets registered at a single peer, all other peers get notified and update their corresponding stores of peers.
    - Create an endpoint to query registered nodes on your network.
    - Create an endpoint to remove a single node or a set of nodes from your network. Similarly as above, the other nodes should be notified about that nodes were removed. However, the mode of operation is now, that each node distributes a list of nodes that it thinks are registered to the network. All nodes should agree on keeping the shortest list of nodes.
  * Add gracefull shutdown to each of your p2p nodes, so that no requests get lost unintentionally. 
    
    

# Reflection


  * What is the difference between a URL and a URI?
  * What is the difference between integrating systems via RPC and REST APIs?
  * Does each document in REST requests and reponses necessarily have a direct counterpart in a system's database?
  * What is a good datastructure to keep a list of known peers on a single node in a P2P network?
  * What is the difference between a client-server and a peer-to-peer architecture?
  * What does it mean that an HTTP PUT is idempotent?
  * What kind of web service did you built when integrating the Github API and the Open Weather Map API?
  